In [1]:
#notebook to calculate accuracy of the models and compare to benchmarks community votes and betting odds
#import necessary packages
import pandas as pd
import numpy as np

In [31]:
#function to calculate rps score
def rps(predictions, observed):
    """Vectorized version of Ranked Probability Score.
    A lower value is a better score.
    From: Colin Catlin, https://syllepsis.live/2022/01/22/ranked-probability-score-in-python/

    Args:
        predictions (pd.DataFrame): each column is an outcome category
            with values as the 0 to 1 probability of that category
        observed (pd.DataFrame): each column is an outcome category
            with values of 0 OR 1 with 1 being that category occurred
    """
    assert (
        predictions.shape == observed.shape
    ), "prediction and observed array shapes must match"
    ncat = predictions.shape[1] - 1
    return (
        np.sum(
            (np.cumsum(predictions, axis=1) - np.cumsum(observed, axis=1)) ** 2, axis=1
        ) / ncat
    )
predictions_df = pd.DataFrame({
    'H': [1, 0.9, 0.8, 0.5, 0.35, 0.6, 0.6, 0.6, 0.5, 0.55],
    'D': [0, 0.1, 0.1, 0.25, 0.3, 0.3, 0.3, 0.1, 0.45, 0.1],
    'A': [0, 0, 0.1, 0.25, 0.35, 0.1, 0.1, 0.3, 0.05, 0.35],
    "model": np.tile(["a", "b"], 5),
    "match": np.repeat(np.arange(1, 6), 2),
    "outcome": np.repeat(["H", "H", "D", "H", "H"], 2),
})
expected_result = [0, 0.005, 0.025, 0.15625, 0.1225, 0.185, 0.085, 0.125, 0.12625, 0.1625]
predictions = predictions_df[["H", "D", "A"]]
observed = pd.get_dummies(predictions_df["outcome"]).reindex(
    columns=predictions.columns, fill_value=0
)
rps_result = rps(predictions, observed).round(5)


#print(predictions)
#print(f"do results match expected: {all(expected_result == rps_result)}")

#print(np.mean(rps_result))

In [33]:
#calculate rps score for betting odds
PL_20_21 = pd.read_csv('Data/2020-21/Fixtures/E_20_21.csv')
PL_21_22 = pd.read_csv('Data/2021-22/Fixtures/E_21_22.csv')
PL_20_21['B365H'] = 1/PL_20_21['B365H']
PL_20_21['B365D'] = 1/PL_20_21['B365D']
PL_20_21['B365A'] = 1/PL_20_21['B365A']
PL_20_21['SUM'] = PL_20_21['B365H'] + PL_20_21['B365D'] + PL_20_21['B365A']
PL_20_21['B365H'] = PL_20_21['B365H']/PL_20_21['SUM']
PL_20_21['B365D'] = PL_20_21['B365D']/PL_20_21['SUM']
PL_20_21['B365A'] = PL_20_21['B365A']/PL_20_21['SUM']
PL_21_22['B365H'] = 1/PL_21_22['B365H']
PL_21_22['B365D'] = 1/PL_21_22['B365D']
PL_21_22['B365A'] = 1/PL_21_22['B365A']
PL_21_22['SUM'] = PL_21_22['B365H'] + PL_21_22['B365D'] + PL_21_22['B365A']
PL_21_22['B365H'] = PL_21_22['B365H']/PL_21_22['SUM']
PL_21_22['B365D'] = PL_21_22['B365D']/PL_21_22['SUM']
PL_21_22['B365A'] = PL_21_22['B365A']/PL_21_22['SUM']
probabilities_20 = PL_20_21[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'FTR']]
probabilities_20 = probabilities_20.rename(columns={'B365H':'H','B365D':'D','B365A':'A'})
predictions_20 = probabilities_20[["H", "D", "A"]]
observed_20 = pd.get_dummies(probabilities_20["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)

probabilities_21 = PL_21_22[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'FTR']]
probabilities_21 = probabilities_21[-350:]
probabilities_21 = probabilities_21.rename(columns={'B365H':'H','B365D':'D','B365A':'A'})
predictions_21 = probabilities_21[["H", "D", "A"]]
observed_21 = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)



rps_result_21 = rps(predictions_21, observed_21).round(5)
rps_result_20 = rps(predictions_20, observed_20).round(5)

rps_betting_odds = np.mean(rps_result_21)
print(rps_betting_odds)

0.19134634285714286


In [35]:
#calculate rps score for community votes
PL_21_22.replace(['Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Norwich', 'Sheff Utd', 'Spurs'],
                 ['Leeds United', 'Leicester City', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
                  'Norwich City', 'Sheffield Utd', 'Tottenham'], inplace=True)
PL_21_22['game_id'] = PL_21_22['HomeTeam'] + PL_21_22['AwayTeam']
PL_21_22['HomeTeam'].drop_duplicates()
transfermarkt_probs_21 = pd.read_csv('Data/2021-22/Fixtures/tranfermarkt_21_probs.csv')
test = pd.merge(PL_21_22, transfermarkt_probs_21, on='game_id', how='left')

predictions = test[["H", "D", "A"]]
observed = pd.get_dummies(test["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result_community = rps(predictions, observed).round(5)

rps_community_votes = np.mean(rps_result_community)

print('RPS score community votes:', rps_community_votes)

RPS score community votes: 0.22040236842105265


In [37]:
#calculate rps score for rf model benchmark
probs_benchmark = pd.read_csv('Data/Total/probs_benchmark.csv')

predictions = probs_benchmark[["H", "D", "A"]]
observed = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result = rps(predictions, observed).round(5)

rps_rf_benchmark = np.mean(rps_result)
print(rps_rf_benchmark)

0.2158239210526316


In [38]:
#calculate rps score for rf model benchmark
probs_player_stats= pd.read_csv('Data/Total/probs_player_stats.csv')

predictions = probs_player_stats[["H", "D", "A"]]
observed = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result = rps(predictions, observed).round(5)

rps_rf_player_stats = np.mean(rps_result)
print(rps_rf_player_stats)

0.21415544736842104


In [39]:
print('RPS score random forest benchmark:', rps_rf_benchmark)
print('RPS score random forest incl player stats:', rps_rf_player_stats)
print('RPS score community votes:', rps_community_votes)
print('RPS score betting odds:', rps_betting_odds)

RPS score random forest benchmark: 0.2158239210526316
RPS score random forest incl player stats: 0.21415544736842104
RPS score community votes: 0.22040236842105265
RPS score betting odds: 0.19134634285714286


In [52]:

#probs_player_stats_info.to_excel('probs_check.xlsx')



In [53]:
probs_player_stats_info

,Unnamed: 0,HomeTeam,AwayTeam,FTR,H,D,A,rf_model_vote,result
0,0,Crystal Palace,Tottenham,H,0.631022,0.200140,0.168838,0,0
1,1,Arsenal,Norwich City,H,0.660644,0.136281,0.203074,0,0
2,2,Brentford,Brighton,A,0.151899,0.674000,0.174101,1,1
3,3,Leicester City,Manchester City,A,0.112982,0.771997,0.115021,1,1
4,4,Manchester Utd,Newcastle Utd,H,0.848845,0.075893,0.075262,0,0
...,...,...,...,...,...,...,...,...,...
345,345,Crystal Palace,Manchester Utd,H,0.259396,0.378656,0.361948,2,0
346,346,Leicester City,Southampton,H,0.626677,0.173323,0.200000,0,0
347,347,Liverpool,Wolves,H,0.604902,0.264070,0.131028,0,0
348,348,Manchester City,Aston Villa,H,0.629223,0.209536,0.161241,0,0
